In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [2]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [3]:
# Data paths
data_dir = r"C:\Users\TAHMID HOSSAIN\Desktop\data"  # Replace with your dataset path


In [4]:
# Data preprocessing
transform = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to VGG-19 input size
        transforms.RandomHorizontalFlip(),  # Data augmentation
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],  # ImageNet mean
            std=[0.229, 0.224, 0.225]   # ImageNet std
        )
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
}

In [5]:
# Load datasets
data = {
    "train": datasets.ImageFolder(root=f"{data_dir}/train", transform=transform["train"]),
    "val": datasets.ImageFolder(root=f"{data_dir}/val", transform=transform["val"])
}

dataloaders = {
    "train": torch.utils.data.DataLoader(data["train"], batch_size=64, shuffle=True),
    "val": torch.utils.data.DataLoader(data["val"], batch_size=64, shuffle=False)
}

In [6]:
# Initialize the VGG-19 model
vgg19 = models.vgg19(pretrained=True)


C:\Users\TAHMID HOSSAIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\TAHMID HOSSAIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Modify the classifier for medical classification (e.g., 2 classes)
num_features = vgg19.classifier[6].in_features
vgg19.classifier[6] = nn.Linear(num_features, len(data["train"].classes))  # Adjust to your dataset
vgg19 = vgg19.to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg19.parameters(), lr=0.0001)


In [9]:
# Training loop
def train_model(model, dataloaders, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 20)
        
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data[phase])
            epoch_acc = running_corrects.double() / len(data[phase])

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    return model

In [10]:

# Train the model
model = train_model(vgg19, dataloaders, criterion, optimizer, num_epochs=20)


Epoch 1/20
--------------------
train Loss: 0.3833 Acc: 0.8561
val Loss: 0.2306 Acc: 0.9092
Epoch 2/20
--------------------
train Loss: 0.1207 Acc: 0.9580
val Loss: 0.1404 Acc: 0.9420
Epoch 3/20
--------------------
train Loss: 0.0760 Acc: 0.9744
val Loss: 0.0793 Acc: 0.9771
Epoch 4/20
--------------------
train Loss: 0.0606 Acc: 0.9818
val Loss: 0.0517 Acc: 0.9840
Epoch 5/20
--------------------
train Loss: 0.0449 Acc: 0.9870
val Loss: 0.0437 Acc: 0.9840
Epoch 6/20
--------------------
train Loss: 0.0343 Acc: 0.9890
val Loss: 0.0403 Acc: 0.9863
Epoch 7/20
--------------------
train Loss: 0.0616 Acc: 0.9818
val Loss: 0.0507 Acc: 0.9855
Epoch 8/20
--------------------
train Loss: 0.0294 Acc: 0.9912
val Loss: 0.0777 Acc: 0.9741
Epoch 9/20
--------------------
train Loss: 0.0214 Acc: 0.9930
val Loss: 0.0542 Acc: 0.9886
Epoch 10/20
--------------------
train Loss: 0.0239 Acc: 0.9921
val Loss: 0.0232 Acc: 0.9893
Epoch 11/20
--------------------
train Loss: 0.0081 Acc: 0.9974
val Loss: 0.021

In [4]:
# Save the trained model
torch.save(model.state_dict(), "vgg19_medical.pth")
print("Model saved!")


Model saved!


In [32]:
import torch
import torchvision.models as models
import torch.nn as nn

# Reinitialize the model (with the same architecture as before)
model = models.vgg19(pretrained=False)  # Don't load ImageNet weights here
num_features = model.classifier[6].in_features  # Get the number of input features to the final layer
model.classifier[6] = nn.Linear(num_features, 2)  # Modify the classifier layer for your dataset (e.g., 2 classes)

# Load the saved state_dict with strict=False
model.load_state_dict(torch.load("vgg19_medical.pth"), strict=False)

# Move the model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Model loaded successfully!")


Model loaded successfully!


C:\Users\TAHMID HOSSAIN\AppData\Local\Temp\ipykernel_7832\988190324.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("vgg19_medical.pth"